In [644]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3
import re
import collections
from collections import defaultdict
from dateutil import parser
from datetime import datetime
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import csv
import re

In [2]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
s3r = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)

In [3]:
d_all = es.search(index = index_name2, query={
                                     "query": {"match_all": {}},
                                     "size":  4
                                     })

In [423]:
def validation_check(user,row):
    educations = get_educations(user)
    match_any = False
    for edu in educations:
        success = True
        for value in ["institution"]:#, "degree_type", "years_low", "years_high"]: 
            resume_value = str(edu[value]).lower()
            validation_value = str(row[value]).lower()
            print(resume_value)
            print('---')
            print(validation_value)
            print(fuzz.ratio(resume_value,validation_value))
            fuzz_score = fuzz.ratio(resume_value,validation_value)
            if fuzz_score < 70:
                success = False
        if success:
            match_any = True
        if success:
            return True
        else:
            return False

In [424]:
def run_test(es, validation_csv_filename):
    tdf = pd.read_csv(validation_csv_filename)
    #tdf = tdf.head(15)
    tdf.columns = [i.lower() for i in tdf.columns]
    if tdf.shape[0] == 0:
        return 0.0
    tdf.sort_values('id', inplace=True)
    failures = []
    success_count = 0
    for r in range(tdf.shape[0]):
        row = tdf.iloc[r].to_dict()
        id = row['id']
        print(id)
        user_record = es.search(index=index_name2,query={"query" : {"match":{"_id": id}}})
        if len(user_record['hits']['hits']) > 0:
            user = user_record['hits']['hits'][0]
            is_valid = validation_check(user,row)
            if is_valid:
                success_count += 1
            else:
                failures.append({"correct": row, "user_record": get_educations(user)})
    return success_count / tdf.shape[0], failures

In [425]:
success, failures = run_test(es, 'Validations.csv')
print(success)

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_8harrison+dataskeptic@gmail.com_Eric Harrison - Resume Professional 2017 December.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_8harrison+dataskeptic@gmail.com_Eric Harrison - Resume Professional 2017 December.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_Adpeters@gmail.com_Data Scientist Resume 2017 New.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_Adpeters@gmail.com_Data Scientist Resume 2017 New.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_Bhautandel@yahoo.com_CV_WP.pdf.txt
['University of Leicester', 'Maharashtra Institute']
university of leicester
---
maharashtra institute of technolo

['Linfield College', 'Galvanize, Inc', 'University of Hawaii']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_bowersa@hawaii.edu_resume-AmandaBowers02.06.18.pdf.txt
['Linfield College', 'Galvanize, Inc', 'University of Hawaii']
university of hawaii
---
university of hawaii-manoa
87
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_bshandro@gmail.com_Bret_SHANDRO_RESUME.pdf.txt
['International Snow Science Workshop', 'Avalanche Association', 'University of Alberta', 'IPENZ EIT', 'APPEGA', 'IPENZ', 'Simon Fraser University']
university of alberta
---
simon fraser university
45
['International Snow Science Workshop', 'Avalanche Association', 'University of Alberta', 'IPENZ EIT', 'APPEGA', 'IPENZ', 'Simon Fraser University']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_bshandro@gmail.com_Bret_SHANDRO_R

['United States Army', 'Canadian College of Family Physicians', 'CCFP-EM', 'Canadian Medical Protective Association', 'Canadian Medical Association', 'Hamilton Ontario', 'Family Medicine University', 'Saskatchewan Medical Association', 'University of Saskatchewan Professional Development', 'Emergency Medicine University', 'Medicine McMaster University', 'Surrey Memorial Hospital', 'Royal Military College of Canada Work Experience Emergency Physician']
family medicine university
---
medicine mcmaster university, hamilton ontario
53
['United States Army', 'Canadian College of Family Physicians', 'CCFP-EM', 'Canadian Medical Protective Association', 'Canadian Medical Association', 'Hamilton Ontario', 'Family Medicine University', 'Saskatchewan Medical Association', 'University of Saskatchewan Professional Development', 'Emergency Medicine University', 'Medicine McMaster University', 'Surrey Memorial Hospital', 'Royal Military College of Canada Work Experience Emergency Physician']
https:/

["King's College", 'University of London', 'WMS', 'MSc Behavioural Data Science', 'Goldsmiths University', 'University of Amsterdam', 'SPSS']
university of london
---
university of amsterdam, amsterdam, netherlands
51
["King's College", 'University of London', 'WMS', 'MSc Behavioural Data Science', 'Goldsmiths University', 'University of Amsterdam', 'SPSS']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_jared_colbert@att.net_Colbert Jared Resume.pdf.txt
[]
[]
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_jared_colbert@att.net_Colbert Jared Resume.pdf.txt
[]
[]
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_jcas81@gmail.com_CV_JCAS.pdf.txt
['Imperial College London', 'United Kingdom', 'Universidade Nova de Lisboa', 'WWW']
['Imperial College London', 'United Kingdom', 'Universidade Nova de Lisboa',

['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_lsarath1703@gmail.com_Sarath Laghumarapu Resume.pdf.txt
['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_lsarath1703@gwu.edu_Sarath Laghumarapu Resume.pdf.txt
['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
['Maybank', 'S&P', 'Remotech Engineers Ltd.', 'GWU', 'National Association of Asian MBAs', 'MassMutual']
https://s3.amazonaws.com/careers.datas

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_navid.nobani@gmail.com_Navid_Nobani_JAN18_fb.pdf.txt
['Pirelli', 'Milan Italy', 'Python', 'Milan', 'Hormozgan University', 'Miandasht Mabna', 'Bandar Abbas Iran', 'Prometeon Tyre Group', 'Foulad Mahan Co.', 'MIANDASHT MABNA CO.', 'Pirelli Industrial', 'Italy', 'Politecnico di Milano', 'Bandar Abbas', 'Prometeon Tyre Group Milan']
hormozgan university
---
hormozgan university, bandar abbas
74
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_navid.nobani@gmail.com_Navid_Nobani_JAN18_fb.pdf.txt
['Pirelli', 'Milan Italy', 'Python', 'Milan', 'Hormozgan University', 'Miandasht Mabna', 'Bandar Abbas Iran', 'Prometeon Tyre Group', 'Foulad Mahan Co.', 'MIANDASHT MABNA CO.', 'Pirelli Industrial', 'Italy', 'Politecnico di Milano', 'Bandar Abbas', 'Prometeon Tyre Group Milan']
hormozgan university
---
politecnico di milano, mil

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Accept_Resume_Data_Skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Accept_Resume_Data_Skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Resume Data skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Resume Data skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Resume Data skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_pereira.inez@gmail.com_Resume_Data_Skeptic.pdf.txt
https://s3.ama

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_randomdog19@gmail.com_Resume - Data Skeptic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_randompi@gmail.com_Barnhart Resume 2018.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_reginaryann@gmail.com_Regina_Ryan_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_sapkotaujwal@gmail.com_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_career_page1_2018-03_sapkotaujwal@gmail.com_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-03text/resumes_care

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_laura.d.manolache@gmail.com_LManolache_cv.pdf.txt
['DESTEC Universita di Pisa', 'Technical University of Civil Engineering Bucharest', 'University of Civil Engineering Bucharest', 'Concordia University', 'ROMANIA', 'ROMANIA BSc of Engineering in Civil, Industrial and Agricultural Engineering', 'Montreal CANADA', 'TUCEB', 'University of Pisa', 'CERN']
technical university of civil engineering bucharest
---
university of pisa, italy
45
['DESTEC Universita di Pisa', 'Technical University of Civil Engineering Bucharest', 'University of Civil Engineering Bucharest', 'Concordia University', 'ROMANIA', 'ROMANIA BSc of Engineering in Civil, Industrial and Agricultural Engineering', 'Montreal CANADA', 'TUCEB', 'University of Pisa', 'CERN']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/career_page1/2018-04text/resumes_career_page1_2018-04_laura.d.manolache@gmail.com_LManolach

['Jilin University Changchun', 'Stony Brook University', 'University of Finance and Economics']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_XiaokeFeiResume.pdf.txt
['Jilin University Changchun', 'Stony Brook University', 'University of Finance and Economics']
jilin university changchun
---
jilin university
76
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_XiaokeFeiResume.pdf.txt
['Jilin University Changchun', 'Stony Brook University', 'University of Finance and Economics']
jilin university changchun
---
central university of finance and economics
49
['Jilin University Changchun', 'Stony Brook University', 'University of Finance and Economics']
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Xiaoxi_Xie.pdf.txt
['University of California']
university of california
---
university of california, los angeles, los angeles, ca
62
['University of California']
https

In [422]:
failures

[{'correct': {'degree level': 'BS',
   'degree type': 'Chemistry & Economics',
   'id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_YM_PARK_RESUME.pdf.txt',
   'institution': 'Sogang University',
   's3 key': 'resumes/unsourced/text/resumes_unsourced_YM_PARK_RESUME.pdf.txt',
   'year high': '2013',
   'years low': '2007'},
  'user_record': [{'degree_type': 'BS',
    'institution': 'University of Wisconsin',
    'years_high': '2016',
    'years_low': '2007'}]},
 {'correct': {'degree level': 'BS',
   'degree type': 'Electrical & Information Engineering',
   'id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_YanbinLu.pdf.txt',
   'institution': 'Huazhong University of Science and Technology',
   's3 key': 'resumes/unsourced/text/resumes_unsourced_YanbinLu.pdf.txt',
   'year high': '2011',
   'years low': '2007'},
  'user_record': [{'degree_type': 'BS',
    'institution': 'University of Houston',
 

In [450]:
def get_educations(parsed_resume):
    educations = []
    education = parsed_resume['_source']['education']
    id = parsed_resume['_id']
    user_record = es.search(index=index_name1,query={"query" : {"match":{"_id": id}}})
    user_record_degree = user_record['hits']['hits'][0]['_source']
    if len(parsed_resume['_source']['education']['entities']) > 0:
        text = [i['Text'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        type = [i['Type'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        start_pos = [i['BeginOffset'] for i in parsed_resume['_source']['education']['entities']['Entities']]
        pos_d = dict(zip(text, start_pos))
        my_dict = defaultdict(list)
        for k, v in zip(type, text):
            my_dict[k].append(v)
        organizations = list(set(my_dict['ORGANIZATION']))
        for organization in organizations:
            school_details = get_school_details(organization, pos_d, user_record_degree, my_dict)
            if school_details is not None:
                educations.append(school_details)
    return  educations

In [451]:
get_educations(user)

university of wisconsin-madison fall 2013 spring 2016
master of science in statistics & master of science in computer sciences, may 2016
coursework
- statistics: mathematical statistics, regression analysis & anova, financial statistics, biostatistics
- computer sciences: machine learning, optimization theory (linear & nonlinear), data structure
sogang university spring 2007 spring 2013
bachelor of science in chemistry & bachelor of science in economics, august 2013
highest ranked student in the graduating class of chemistry department


[{'degree_type': 'BS',
  'institution': 'University of Wisconsin',
  'years_high': '2016',
  'years_low': '2007'}]

In [176]:
def parse_date(s):
    result = ""
    for ch in s:
        if ch.isdigit():
            result += ch
    return result

In [407]:
def get_school_details(orgname, pos_d, user_record_degree, my_dict):
    org_pos = pos_d[orgname]
    if orgname.lower().find('university') != -1:
        years = my_dict['DATE']
        years = [parse_date(i) for i in years]
        years_low = -1
        years_high = -1
        if len(years) > 0:
            years_low = min(years)
            years_high = max(years)
        degree_type = find_degree_type(org_pos,user_record_degree)
        edu_info = {"institution": orgname,"degree_type":degree_type, "years_low":years_low,"years_high":years_high}
        return edu_info
    else: 
        return None

In [409]:
def find_degree_type(univ_pos,user_record):
    values = {"bachelor":'BS',"master":'MS','phd':'phD','B.S.':'BS'}
    for k in user_record:
        if 'education' in k:
            degree_d = {}
            for key in values.keys():
                degree_pos = user_record['education'].lower().find(key)
                degree_d[degree_pos] = key
            degree_type = degree_d.get(univ_pos, degree_d[min(degree_d.keys(), key=lambda k: abs(k-univ_pos))])
            abbr_degree = values[degree_type]
    return abbr_degree

In [145]:
tdf = pd.read_csv('Validations.csv')

In [129]:
#tdf = tdf.head()

In [641]:
url = 'https://raw.githubusercontent.com/endSly/world-universities-csv/master/world-universities.csv'
import pandas as pd
data = pd.read_csv(url)
data.columns = ['country','school','web']
data = data[['school']]
data['school'] = [i.lower() for i in data['school']]

In [633]:
data.tail()

,school
9357,solusi university
9358,university of zimbabwe
9359,women's university in africa
9360,zimbabwe ezekiel guti university
9361,zimbabwe open university


In [561]:
user_record_degree=es.search(index=index_name1,query={"query" : {"match":{"_id": 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_YM_PARK_RESUME.pdf.txt'}}})['hits']['hits'][0]['_source']

In [596]:
def get_school_name(user_record_degree):
    content = user_record_degree['education'].lower()
    matches = []
    for school in range(len(data['school'])):
        match = content.find(data.iloc[school]['school'])
        leng = len(data.iloc[school]['school'])
        matches.append(match)
    indexes = [ n for n,i in enumerate(matches) if i>-1 ][0]
    leng = len(data.iloc[indexes][0])
    univ_pos = [i for i in matches if i != -1]
    schools = []
    for pos in univ_pos:
        school_name = content[pos:pos+leng]
    schools.append(school_name)
    return schools
   
    
    

In [597]:
get_school_name(user_record_degree)

['sogang university']

In [599]:
print(user_record_degree['education'])

University of Wisconsin-Madison Fall 2013 Spring 2016
Master of Science in Statistics & Master of Science in Computer Sciences, May 2016
Coursework
- Statistics: Mathematical Statistics, Regression Analysis & ANOVA, Financial Statistics, Biostatistics
- Computer Sciences: Machine Learning, Optimization Theory (Linear & Nonlinear), Data Structure
Sogang University Spring 2007 Spring 2013
Bachelor of Science in Chemistry & Bachelor of Science in Economics, August 2013
Highest Ranked Student in the Graduating Class of Chemistry Department


In [650]:
user_record_degree['education']

'University of Wisconsin-Madison Fall 2013 Spring 2016\nMaster of Science in Statistics & Master of Science in Computer Sciences, May 2016\nCoursework\n- Statistics: Mathematical Statistics, Regression Analysis & ANOVA, Financial Statistics, Biostatistics\n- Computer Sciences: Machine Learning, Optimization Theory (Linear & Nonlinear), Data Structure\nSogang University Spring 2007 Spring 2013\nBachelor of Science in Chemistry & Bachelor of Science in Economics, August 2013\nHighest Ranked Student in the Graduating Class of Chemistry Department'

In [649]:
data.head()

,school
0,abu dhabi university
1,ajman university of science and technology
2,alain university of science and technology
3,al ghurair university
4,alhosn university


In [615]:
a = data.loc[data['school'].apply(lambda x: x.find('wisconsin') >=0)]

In [608]:
def remove_spe_chara(string):
    return re.sub(r'\W+', ' ', string)

In [618]:
#data.loc[data['school'].apply(lambda x: x.find('wisconsin') >=0)]

In [ ]:
data['school']=[re.sub('&', 'and', i) for i in data['school']]

In [634]:
data['school'] = data['school'].apply(lambda x: remove_spe_chara(x).lower())

In [637]:
data.head()

,school
0,abu dhabi university
1,ajman university of science & technology
2,alain university of science and technology
3,al ghurair university
4,alhosn university


In [647]:
data['school']=[re.sub('&', 'and', i) for i in data['school']]
